In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorboardX import SummaryWriter
import time
import numpy as np
from PIL import Image

In [ ]:
# Configuration Variables
MEAN_PREPRO = [0.485, 0.456, 0.406]
STD_PREPRO = [0.229, 0.224, 0.225]
RESIZE_PREPRO = 256,256
RESIZE_DRAW = 256,256

TRAIN_BATCH_SIZE = 128
TRAIN_SHUFFLE = True
TRAIN_NUM_WORKERS = 8
TRAIN_PIN_MEMORY = True

VAL_BATCH_SIZE = 512
VAL_SHUFFLE = False
VAL_NUM_WORKERS = 4
VAL_PIN_MEMORY = True

INITIAL_LR = 1e-4
DEVICE_ID = 1

DATA_DIR = '/data/porn/binary/'
TRAINSET_ROOT = f'{DATA_DIR}/train'
TESTSET_ROOT = f'{DATA_DIR}/test'

TENSORBOARD_DIR = '/data/porn/tensorboard/resnext101/'

In [ ]:
normalize = transforms.Normalize(
        mean=MEAN_PREPRO, std=STD_PREPRO
    )
prepro = transforms.Compose(
    [
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]
)
prepro_val = transforms.Compose(
    [transforms.Resize(RESIZE_PREPRO), transforms.ToTensor(), normalize]
)

prepro_draw = transforms.Compose(
    [transforms.Resize(RESIZE_DRAW), transforms.ToTensor()]
)


In [82]:
class validation_dataset(ImageFolder):
    def __init__(self, root, transform, target_transform=None):
        super().__init__(root, transform=transform, target_transform=target_transform)
        
    def __getitem__(self, index):
        data, label = super().__getitem__(index)
        return data, label, index

In [84]:
train_dataset = ImageFolder(TRAINSET_ROOT, transform=prepro)
val_dataset = validation_dataset(TESTSET_ROOT, transform=prepro_val)
print(train_dataset.classes)

['nsfw', 'safe']


In [85]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=VAL_SHUFFLE, num_workers=VAL_NUM_WORKERS, pin_memory=VAL_PIN_MEMORY, drop_last=False)

In [86]:
model = models.resnext101_32x8d(pretrained=True)
model.fc = torch.nn.Linear(2048, 1)
DEVICE = f"cuda:{DEVICE_ID}"
model = model.to(DEVICE)
for p in model.parameters():
    p.requires_grad=False
for p in model.fc.parameters():
    p.requires_grad=True

In [87]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR)
criterion = nn.SoftMarginLoss() # TODO change it to crossentropy

In [88]:
def plot_cm(classes, cm):
    fig, ax = plt.subplots(figsize=(len(classes),len(classes)))  
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt="d")
    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix') 
    ax.xaxis.set_ticklabels(classes,rotation=90)
    ax.yaxis.set_ticklabels(classes,rotation=0)
    return fig

In [89]:
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    on_iteration=None,
):
    model = model.train()
    end = time.time()
    print("Start Training")
    avg_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"{i/len(train_loader) * 100 : 2.2f}%", end="\r")
        iteration_time = time.time()
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        if on_iteration is not None:
            on_iteration(iteration=i+epoch*len(train_loader), loss=loss, y_pred=outputs, y_true=labels)     
    return avg_loss/len(train_loader)

In [90]:
def validate(val_loader, 
             model, 
             criterion,
             print_freq=1000):
    model = model.eval()
    y_pred, y_true, proba_pred, indexes = [], [], [], []
    avg_loss = 0
    for i, (inputs, labels, image_indexes) in enumerate(val_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        with torch.no_grad():
            outputs = model(inputs)
            avg_loss += criterion(outputs, labels).item()
        proba_pred.append(outputs.cpu().clone())
        _, indices_max = torch.max(outputs.cpu(), 1)
        y_pred.append(indices_max)
        y_true.append(labels.cpu().clone())
        indexes.append(image_indexes.clone())
    return {"loss":avg_loss/len(val_loader), 
            "class_prediction":torch.cat(y_pred), 
            "ground_truth":torch.cat(y_true), 
            "probabilities":torch.cat(proba_pred), 
            "images_index":torch.LongTensor(torch.cat(indexes))}

In [91]:
def draw_max_error(epoch, val_res, n=5):
    # max false positive
    fp = (val_res["class_prediction"] == 0) & (val_res["ground_truth"] == 1)
    values, images = [], []
    for i in range(len(fp)):
        if fp[i]:
            values.append(val_res["probabilities"][i][1])
            images.append(val_dataset.imgs[val_res["images_index"][i]][0])
    _, max_indexes = torch.sort(torch.Tensor(values), descending=True)
    for i in range(n):
        logger.add_image(f"Eval/FalsePositive/{i}", prepro_draw(Image.open(images[max_indexes[i]])), epoch)
        
    #false negative
    fn = (val_res["class_prediction"] == 1) & (val_res["ground_truth"] == 0)
    values, images = [], []
    for i in range(len(fn)):
        if fn[i]:
            values.append(val_res["probabilities"][i][0])
            images.append(val_dataset.imgs[val_res["images_index"][i]][0])
    _, max_indexes = torch.sort(torch.Tensor(values), descending=True)
    for i in range(n):
        logger.add_image(f"Eval/FalseNegative/{i}", prepro_draw(Image.open(images[max_indexes[i]])), epoch)

In [ ]:
def compute_regression(pred, gt):
    c = 0
    TRUE_POSITIVE = gt == 1
    PRED_POSITIVE = gt > 0.5
    VP = TRUE_POSITIVE & PRED_POSITIVE
    FN = TRUE_POSITIVE & not PRED_POSITIVE
    VN = not TRUE_POSITIVE & not PRED_POSITIVE
    FP = not TRUE_POSITIVE & PRED_POSITIVE
    PRECISION = float(VP.sum()) / (PRED_POSITIVE.sum())
    RECALL = float(VP.sum()) / (VP.sum() + FN.sum())
    return recall, precision

In [92]:
def compute_score(val_res):
    VP = (val_res["class_prediction"] == 0) & (val_res["ground_truth"] == 0)
    FP = (val_res["class_prediction"] == 0) & (val_res["ground_truth"] == 1)
    FN = (val_res["class_prediction"] == 1) & (val_res["ground_truth"] == 0)
    vp = float(VP.sum())
    fp = FP.sum()
    fn = FN.sum()
    precision = vp / (vp + fp)
    recall = vp / (vp + fn)
    return precision, recall

In [ ]:
def validation_logs(epoch, validation_res):
    logger.add_scalar("Loss/Avg_Val", validation_res["loss"], epoch)
    logger.add_scalar("Eval/Accuracy", (validation_res["class_prediction"]==validation_res["ground_truth"]).sum()/float(len(validation_res["ground_truth"])), epoch)
    precision, recall = compute_score(validation_res)
    logger.add_scalar("Eval/Recall", recall, epoch)
    logger.add_scalar("Eval/Precision", precision, epoch)
    probas = [x[validation_res["ground_truth"][i]] for i,x in enumerate(validation_res["probabilities"])]
    logger.add_pr_curve("Eval/Prec_recall", validation_res["ground_truth"], probas, epoch)
    draw_max_error(epoch, validation_res)
    cm = confusion_matrix(validation_res["class_prediction"], validation_res["ground_truth"])
    cm_image = plot_cm(train_dataset.classes, cm)
    logger.add_figure("Eval/Confusion Matrix", cm_image, epoch)

In [ ]:
def on_iteration_logs(iteration, loss, y_pred, y_true):
    l = loss.item()
    if iteration%200 == 0:
        logger.add_scalar("Loss/Train", l, iteration)
        print(
                f"{iteration}/{len(train_loader)} \t"
                f"Loss {l}"
            )

In [ ]:
logger = SummaryWriter(TENSORBOARD_DIR)

In [ ]:
for i in range(10):
    val_res = validate(val_loader, model, criterion)
    validation_logs(i, val_res)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

In [ ]:
for i, child in enumerate(model.children()):
    if i > 6:
        for p in child.parameters():
            p.requires_grad=True

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE//2, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)

In [ ]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.1)

In [ ]:
for i in range(10,40):
    val_res = validate(val_loader, model, criterion)
    validation_logs(i, val_res)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

In [ ]:
DEVICE = "cuda:2"
model = model.to(DEVICE)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE//4, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.1)
for i, child in enumerate(model.children()):
    if i > 3:
        for p in child.parameters():
            p.requires_grad=True
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.01)

In [ ]:
for i in range(40,80):
    val_res = validate(val_loader, model, criterion)
    validation_logs(i, val_res)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)